<a href="https://colab.research.google.com/github/card323/torrent_crwaler/blob/main/torrent_crawler.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Creating a Python Web Crawler

**Understanding the Task:**
We'll create a Python script that:
1. Takes a website URL as input.
2. Crawls the website, following links.
3. Extracts relevant data from the pages.
4. Outputs the data in a structured format (e.g., JSON, CSV).

**Required Libraries:**
* **requests:** For making HTTP requests to fetch web pages.
* **Beautiful Soup:** For parsing HTML content.
* **urllib.parse:** For URL manipulation.

**Code:**

In [23]:
import requests
import re
from bs4 import BeautifulSoup
from urllib.parse import urljoin, quote
import urllib.parse
from datetime import date

def crawl_website(url,blocklist=[], max_depth=1):
    """Crawls a website and extracts data.

    Args:
        url (str): The URL of the website to crawl.
        blocklist (list): A list of URLs to avoid crawling.
        max_depth (int): The maximum depth to crawl.
    """

    visited_urls = set()
    data = []

    def crawl_page(url, depth):
        if url in visited_urls or any(re.match(pattern, url) for pattern in blocklist)  or depth > max_depth:
            return
        visited_urls.add(url)

        try:
            print(f"crawling {url}")
            response = requests.get(url)
            response.raise_for_status()  # Raise an exception for error status codes

            soup = BeautifulSoup(response.content, 'html.parser')

            elements = soup.find_all('div', class_='card mb-3')
            # Extract data from the page (adjust selectors based on website structure)
            for element in elements:
            # Extract data from each element
              title_element = element.find('h5', class_='title is-4 is-spaced')
              title_link = title_element.find('a')
              title = title_link.text.strip() if title_link else None

              pic_element = element.find('img', class_='image')
              pic_url = pic_element['src'] if pic_element else None

              # Extract torrent link from magnet link button
              magnet_link_element = element.find('a', class_='button is-primary is-fullwidth', title='Magnet torrent')
              magnet_link = magnet_link_element['href'] if magnet_link_element else None

              # Extract torrent download link
              download_link_element = element.find('a', class_='button is-primary is-fullwidth', title='Download .torrent')
              download_link = download_link_element['href'] if download_link_element else None
              if download_link and not download_link.startswith('http'):
                  download_link = urllib.parse.urljoin(url, download_link)
              # Add data to list
              data.append({'title': title, 'pic': pic_url, 'magnet_link': magnet_link, 'download_link': download_link})


            # Find and follow links
            for link in soup.find_all('a'):
                href = link.get('href')
                if href and href.startswith('http'):
                    absolute_url = urljoin(url, href)
                    crawl_page(absolute_url,depth+1)
        except requests.exceptions.RequestException as e:
            print(f"Error crawling {url}: {e}")

    crawl_page(url,max_depth)
    return data

# Example usage
with open('actress_names.txt', 'r') as f:
    actress_names = [line.strip() for line in f]
website_urls = [f"https://www.141jav.com/actress/{quote(name)}" for name in actress_names]
new_data = []
blocklist = [r"https://.*\.shopify\.com/.*",r"https://.*\.skilljar\.com/.*"]
for url in website_urls:
    new_data.extend(crawl_website(url, blocklist, 1))

# Load archived data
try:
    with open('archived_data.json', 'r') as f:
        archived_data = json.load(f)
except FileNotFoundError:
    archived_data = []

# Find elements in new_data but not in archived_data
def is_not_in_archived(item, archived_data):
    return not any(item['title'] == a['title'] for a in archived_data)

new_elements = [item for item in new_data if is_not_in_archived(item, archived_data)]

# Update archived data
archived_data.extend(new_elements)

# Save updated archived data
with open('archived_data.json', 'w') as f:
    json.dump(archived_data, f, indent=4)

# Output the extracted data (e.g., as JSON)
import json
today = date.today().strftime("%Y-%m-%d")
filename = f"extracted_data_{today}.json"
# Output the extracted data
with open(filename, 'w') as f:
    json.dump(new_elements, f, indent=4)

def generate_html(data):
    html = "<!DOCTYPE html><html><head><title>Crawled Data</title></head><body>"
    for item in data:
        html += f"<h2>{item['title']}</h2>"
        if item['pic']:
            html += f"<img src='{item['pic']}'><br>"
        if item['download_link']:
            html += f"<a href='{item['download_link']}'>Download</a><br>"
        if item['magnet_link']:
            html += f"<a href='{item['magnet_link']}'>Magnet</a><br>"
        html += "<hr>"
    html += "</body></html>"
    return html

with open(filename, 'r') as f:
    data = json.load(f)

html_content = generate_html(data)

with open('output.html', 'w') as f:
    f.write(html_content)

print("HTML file generated: output.html")

crawling https://www.141jav.com/actress/Nagi%20Hikaru
crawling https://www.141jav.com/actress/Mita%20Marin
crawling https://www.141jav.com/actress/Yoshitaka%20Nene
crawling https://www.141jav.com/actress/Arata%20Arina
crawling https://www.141jav.com/actress/Yatsugake%20Umi
crawling https://www.141jav.com/actress/Miyashita%20Rena
crawling https://www.141jav.com/actress/Sakura%20Momo
crawling https://www.141jav.com/actress/Shidara%20Yuuhi
crawling https://www.141jav.com/actress/Kaede%20Karen
crawling https://www.141jav.com/actress/Neo%20Akari
crawling https://www.141jav.com/actress/Momonogi%20Kana
HTML file generated: output.html


**Explanation**:

1. **Import necessary libraries**: We import requests, BeautifulSoup, urljoin, quote, json, and date.
2. **Define crawl_website function**: This function takes a URL, a blocklist of URLs to avoid, and a maximum depth as input, and crawls the website to extract data.
3. **Define generate_html function**: This function
takes the extracted data and generates an HTML page with the data.
4. **Read actress names from file**: We read actress names from a file named 'actress_names.txt'.
5. **Generate website URLs**: We generate website URLs from the actress names, encoding them for use in URLs.
6. **Load archived data**: We load previously archived data from 'archived_data.json', or initialize an empty list if the file doesn't exist.
7. **Extract new data**: We crawl the websites and extract new data.
8. **Find new elements**: We compare the new data with the archived data and find elements that are not present in the archived data.
9. **Update archived data**: We append the new elements to the archived data.
10. **Save updated archived data**: We save the updated archived data to 'archived_data.json'.
11. **Generate filename with today's date**: We generate a filename for the extracted data with today's date in the format "extracted_data_YYYY-MM-DD.json".
12. **Save extracted data**: We save the extracted data to the generated filename.
13. **Generate HTML**: We generate an HTML page from the extracted data and save it to 'output.html'.


Customization:

* Data extraction: Adjust the selectors (soup.find(...)) in the crawl_website function based on the specific structure of the websites you're crawling.
* Blocklist: Update the blocklist to include any URLs you want to avoid crawling.
Maximum depth: Change the max_depth parameter to control how deep you want to crawl the websites.
* HTML generation: Modify the generate_html function to customize the appearance of the generated HTML page.
* Data filtering: You can add additional logic to filter the extracted data based on specific criteria.
* Error handling: Implement more robust error handling to handle potential issues during the crawling process.
* Rate limiting: Consider adding rate limiting to avoid overwhelming the target websites.